In [6]:
!pip install semantic-kernel==0.9.1b1

In [10]:
import os
import semantic_kernel as sk
import json

from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.planners.action_planner import ActionPlanner

plugins_directory = "plugins"
plugin_names = [plugin for plugin in os.listdir(plugins_directory) if os.path.isdir(os.path.join(plugins_directory, plugin))]
#plugin_names = ["PhilPlugin"]

print("Plugins found: ", plugin_names)

Plugins found:  ['EditingPlugin']


In [11]:
kernel = sk.Kernel()

deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
service_id = "default"

kernel.add_service(
    AzureChatCompletion(service_id=service_id, deployment_name=deployment, endpoint=endpoint, api_key=api_key),
)

for plugin_name in plugin_names:
    print("Loading plugin: ", plugin_name)
    plugin = kernel.import_plugin_from_prompt_directory(plugins_directory, plugin_name)
    plugin.description = "This is the editing plugin."
   
# convert plugin to json string
plugin_json = json.dumps(plugin, default=lambda x: x.__dict__)
print(plugin_json)

   


Loading plugin:  EditingPlugin
{"name": "EditingPlugin", "description": "This is the editing plugin.", "functions": {"ChangeTense": {"metadata": {"name": "ChangeTense", "plugin_name": "EditingPlugin", "description": "Takes a text and changes it to a specified verbal tense.i.e. Present, Past, Future, etc.", "parameters": [{"name": "input", "description": "Text to be processed.", "default_value": "", "type_": "", "is_required": true, "type_object": null}, {"name": "tense", "description": "Tense we want to set the input to.", "default_value": "present", "type_": "", "is_required": true, "type_object": null}], "is_prompt": true, "is_asynchronous": true, "return_parameter": {"name": "return", "description": "The completion result", "default_value": null, "type_": "FunctionResult", "is_required": true, "type_object": null}}, "prompt_template": {"prompt_template_config": {"name": "ChangeTense", "description": "Takes a text and changes it to a specified verbal tense.i.e. Present, Past, Future,

In [3]:
ask = """Change the following sentence to past tense:
I am going to the mall tomorrow to buy some clothes."""

#ask = """Change the following paragraph to past tense:
#Southwest, which flies only Boeing planes, is particularly exposed to the manufacturer’s struggles. Southwest said its plans to cut flying capacity will primarily happen in the second half of the year. The carrier said it is still in the process of reworking plans but will reduce pilot hiring by more than 50% and reduce flight-attendant hiring by more than 60% this year, looking to reduce costs."""

In [12]:
planner = ActionPlanner(kernel, service_id=service_id)

plan = await planner.create_plan(goal=ask)

print(plan.function)

Overwriting plugin "ActionPlanner_Excluded" in collection


metadata=KernelFunctionMetadata(name='ChangeTense', plugin_name='EditingPlugin', description='Takes a text and changes it to a specified verbal tense.i.e. Present, Past, Future, etc.', parameters=[KernelParameterMetadata(name='input', description='Text to be processed.', default_value='', type_='', is_required=True, type_object=None), KernelParameterMetadata(name='tense', description='Tense we want to set the input to.', default_value='present', type_='', is_required=True, type_object=None)], is_prompt=True, is_asynchronous=True, return_parameter=KernelParameterMetadata(name='return', description='The completion result', default_value=None, type_='FunctionResult', is_required=True, type_object=None)) prompt_template=KernelPromptTemplate(prompt_template_config=PromptTemplateConfig(name='ChangeTense', description='Takes a text and changes it to a specified verbal tense.i.e. Present, Past, Future, etc.', template='USE THE INPUT TEXT AND REWRITE IT CHANGING ONLY THE VERBAL TENSE, IF PROVID

In [13]:
results = await plan.invoke(kernel)

In [14]:
print(results)

I was going to the mall the next day to buy some clothes.


## Transform JSON tests

In [18]:
import json

jsonFile = """{
    "content": {
        "Document Title: Examination of the Stability of TAK-188 in Mouse, Monkey, and Human Plasma": "Document Version: 1.0Effective Date: 14 Apr 2023 14:22:20 (GMT-04:00)Signed By : Guo, Yukuang (mne9509) Decision : ApprovedDecision Date : 14 Apr 2023 13:52:28 (GMT-04:00)Role : Approver Purpose : ApprovalMeaning Of Signature : I approve this documentSigned By : Zhu, Xiaochun Sean (UBN7028) Decision : ApprovedDecision Date : 14 Apr 2023 14:22:06 (GMT-04:00)Role : Approver Purpose : ApprovalMeaning Of Signature : I approve this documentSignature Manifestation Page 1 of 1NONCLINICAL STUDY PROTOCOLMetabolism Identification Title:Examination of the Stability of TAK-188 in Mouse, Monkey, and Human Plasma Protocol:TKD-BCS-01991Testing Facility:Takeda Development Center Americas, Inc.35 Landsdowne StreetCambridge, MA 02139 USAStudy Director:Yukuang Guo Ph (312) 722-0479Proposed Study Dates:Experimental Start: 01 April 2023Experimental End: 01 July 2023",
        "INTRODUCTION": "CCR8 expression is upregulated on a highly immunosuppressive subset of regulatory T cells (Tregs) in multiple cancer types. TAK-188 is an Antibody Drug Conjugate (ADC) targeting CCR8. TAK-188 is designed specifically for depleting Tregs and boosting the immune system to produce an antitumoral effect.",
        "STUDY OBJECTIVE": "The objective of this study is to examine the plasma stability of TAK-188 in vitro in fresh mouse, monkey, and human plasma.",
        "REGULATORY GUIDELINES": "",
        "Good Laboratory Practice Compliance": "This study will not be conducted in accordance with the Code of Federal Regulations, Title 21, Part 58: Good Laboratory Practice for Nonclinical Laboratory Studies, issued by the United States Food and Drug Administration. However, this study will be conducted according to standard departmental practices.",
        "Amendments To, and Deviations From, the Protocol": "Any changes to the protocol will be described in a written protocol amendment approved by the Study Director specified in this protocol and recorded in the raw data at Takeda Development Center Americas, Inc. (TDCA) (Cambridge, MA, USA). Any deviations from the protocol will be documented appropriately and listed in the final report.",
        "TESTING FACILITY": "All aspects of this study will be conducted by personnel of TDCA (Cambridge, MA, USA).",
        "STUDY PERSONNEL": "Table 1 lists personnel involved in this study.",
        "PROPOSED STUDY SCHEDULE": "Table 2 provides the proposed study schedule.",
        "Table 2Proposed Study Schedule": "91440052510Study ComponentProposed DateExperimental start date:01 April 2023Experimental end date:01 July 202390525641553",
        "MATERIALS": "Unless otherwise stated, all analytical reagents will be at least of high-performance liquid chromatography (HPLC) reagent grade. Unless otherwise noted, all chemicals, with the exception of the test article (TAK-188) will be purchased from Sigma-Aldrich Corp (St. Louis, MO, USA) or supplied by personnel of TDCA’s Lab Operations group. All reagents, with the exception of the test article, will be less than 5 years old or formulated by personnel of TDCA’s Lab Operations group.",
        "Test Article": "TAK-188 will be supplied by personnel of TDCA’s Sample Bank and stored at approximately −20°C.The test article is early, research-grade material, which has not been completely characterized for identity, purity, and stability, and which does not have a Certificate of Analysis. The identity, storage temperature, lot (batch) numbers, and formulation requirements will be documented in the study file.",
        "Test System": "Fresh mouse, monkey, and human plasma will be purchased from external vendor (information will be provided in the final report).",
        "Reagents and Instruments": "The reagents and instruments used in this study will be specified in the raw data and final report.",
        "ASSAY METHODOLOGY": "",
        "Plasma incubation": "The experimental procedure and final conditions will be specified in the final report. The incubation will be run in triplicates. The incubation conditions are shown in Table 3.",
        "Table 3Incubation Conditions": "91440052480Matrix:Fresh plasmaSpecies:Mouse, monkey, humanTime (hours):0, 4, and 24Concentration (µg/mL):100Incubator Conditions:5% CO2, ~93% humidity, 37°CShaking Rate (rpm):7591440041438rpm: revolutions per minute.",
        "BIOANALYSIS": "The monkey and human samples will be shipped to Frontage Laboratories and mouse samples will be shipped to QPS for bioanalysis (the method and details will be included in the final report) as shown below:Frontage Laboratories Attn: Sample Management 700 Pennsylvania DriveExton, PA 1943Email: Sample_Management@frontagelab.com Phone: (610) 232-0100Attn: Sample Coordinator/Lan Li QPS, LLC3 Innovation WayNewark, DE 19711Tel: (302) 369-5120Fax: (302) 369-5602Email: sample@qps.com",
        "DATA INTERPRETATION AND REPORTING": "The Study Director, with participation by other contributing scientists, as appropriate, will prepare a report describing, but not necessarily be limited to, the following: a summary of the study, study objectives, materials and methods, and results and interpretation, including any appropriate tables, figures, and/or appendices.",
        "REMEASUREMENT AND ADOPTION OF THE DATA": "If the Study Director believes that a certain event (eg, technical error, instrumentation problem, or scientifically unexpected phenomena) may have affected the reliability of the present study, the Study Director may deem it necessary to repeat the experiment(s) in the study. The reasons for repeating any steps of an experiment, and for adoption or exclusion of the results, will be noted in the raw data and the final report.",
        "ARCHIVING": "All study data, including the protocol (with any amendments and deviations), final report, study file, laboratory study notebooks, instrument printouts, and other raw data or materials from the study will be maintained at TDCA according to standard departmental procedures.Title:    Examination of the Stability of TAK-188 in Mouse, Monkey, and Human Plasma",
        "PROTOCOL APPROVAL": "(see electronic signature page)91440042931Yukuang Guo Study Director Scientist IDrug Metabolism and Pharmacokinetics(see electronic signature page)91440041471Xiaochun Sean Zhu Scientific DirectorDrug Metabolism and Pharmacokinetics"
    },
    "headers": {
        "TAK": "TAK-188",
        "TKD": "TKD-BCS-01991"
    }
}"""
body = json.loads(jsonFile)

body["content"]

sections = [(k, v) for k, v in body["content"].items()]

print(len(sections))

headers = body["headers"]

contents = {}

for k, v in sections:
    # append properties to json object    
    contents[k] = v
    
results = []
results.append(("headers", headers))
results.append(("contents", contents))

print(json.dumps(dict(results)))
    

22
{"headers": {"TAK": "TAK-188", "TKD": "TKD-BCS-01991"}, "contents": {"Document Title: Examination of the Stability of TAK-188 in Mouse, Monkey, and Human Plasma": "Document Version: 1.0Effective Date: 14 Apr 2023 14:22:20 (GMT-04:00)Signed By : Guo, Yukuang (mne9509) Decision : ApprovedDecision Date : 14 Apr 2023 13:52:28 (GMT-04:00)Role : Approver Purpose : ApprovalMeaning Of Signature : I approve this documentSigned By : Zhu, Xiaochun Sean (UBN7028) Decision : ApprovedDecision Date : 14 Apr 2023 14:22:06 (GMT-04:00)Role : Approver Purpose : ApprovalMeaning Of Signature : I approve this documentSignature Manifestation Page 1 of 1NONCLINICAL STUDY PROTOCOLMetabolism Identification Title:Examination of the Stability of TAK-188 in Mouse, Monkey, and Human Plasma Protocol:TKD-BCS-01991Testing Facility:Takeda Development Center Americas, Inc.35 Landsdowne StreetCambridge, MA 02139 USAStudy Director:Yukuang Guo Ph (312) 722-0479Proposed Study Dates:Experimental Start: 01 April 2023Experi